In [1]:
import string
from pathlib import Path
from collections import defaultdict
import json
import numpy as np
import math
from typing import List
from functools import total_ordering

class Vertex(object):
    def __init__(self, name, flow):
        self.name = name
        self.flow = flow

    def __repr__(self) -> str:
        return f'V {self.name}, flow rate {self.flow}'

class Edge(object):
    def __init__(self, fromVertex, toVertex):
        self.fromVertex = fromVertex
        self.toVertex = toVertex
    def __repr__(self) -> str:
        return f'{self.fromVertex} -> {self.toVertex}'

class Graph(object):
    def __init__(self, vertices, edges):
        self.vertices = {}
        self.vertices_neigh = {}
        self.edges = []
        for vertex in vertices:
            if vertex[0] not in self.vertices.keys():
                self.vertices[vertex[0]] = Vertex(name=vertex[0], flow=vertex[1])
        for edge in edges:
            assert edge[0] in self.vertices.keys(), f'"{edge[0]}" not in vertices'
            assert edge[1] in self.vertices.keys(), f'"{edge[1]}" not in vertices'
            self.edges.append(Edge(fromVertex=edge[0],toVertex=edge[1]))

        self.compute_neighs()

    def __repr__(self) -> str:
        string = 'Vertices:\n'
        for vertex in self.vertices:
            string += '\t'+str(vertex)+'\n'
        string += '\nEdges:\n'
        for edge in self.edges:
            string += '\t'+str(edge)+'\n'
        return string

    def all_vertices(self):
        return {v.name: v.flow for v in self.vertices.values()}

    def all_edges(self):
        return [(e.fromVertex, e.toVertex) for e in self.edges]

    def compute_neighs(self):
        for v in self.vertices.keys():
            self.vertices_neigh[v] = [e.toVertex for e in self.edges if e.fromVertex==v]

    def vertex_neigh(self, vertex):
        return self.vertices_neigh[vertex]


In [2]:
file = open('./input.txt', 'r')

vertices = []
edges = []

for line in file.readlines():
    line = line.replace('\n','')
    # Valve MO has flow rate=0; tunnels lead to valves QM, ED
    vertex_string, edges_string = line.split(';')

    vertex_name = vertex_string.split('has')[0].replace('Valve','').strip()
    flow_rate = int(vertex_string.split('=')[1])
    vertices.append( (vertex_name, flow_rate) )

    connected = edges_string.split('valve')[1].replace('s ','')
    connected = [e.strip() for e in connected.split(',')]
    for dest in connected:
        edges.append( (vertex_name, dest) )

graph = Graph(vertices, edges)

print(len(graph.all_vertices()))
print(len(graph.all_edges()))
# print(graph)


66
164


In [3]:
# @total_ordering
class State(object):
    def __init__(self, vertex, to_open, time, flow):
        self.vertex = vertex
        self.to_open = to_open
        self.time = time
        self.flow = flow

    def __eq__(self, other):
        return (self.flow == other.flow and self.vertex==other.vertex and self.time==other.time and self.to_open==other.to_open)

    # def __lt__(self, other):
    #     return self.flow < other.flow

    def __repr__(self) -> str:
    
        return f'{self.vertex}_{self.to_open}_F{self.flow}_T{self.time}'

    

In [4]:
def is_better(state, other):
    assert state.vertex == other.vertex
    if state.flow>=other.flow and state.time>=other.time and (state.to_open or (not other.to_open)):
        if (state.flow>other.flow) or (state.time>other.time) or (state.to_open and (not other.to_open)):
            return 2
        else:
            return 1
    else:
        return 0
        
def add_state(states, new_state):

    betters = [is_better(new_state[1], s[1]) for s in states]
    worser = [is_better(s[1], new_state[1]) for s in states]
    removed = []
    best_states = []
    to_add = False
    for i,state in enumerate(states):
        if betters[i]<2:
            best_states.append(state)
        else:
            # print(f'removing {state[1]} because {new_state[1]} is better')
            removed.append(state[0])
    if max(worser)<2:
        best_states.append(new_state)
        to_add = True
    return best_states, to_add, removed

def extract_opened_vertices(idx):
    opened = []
    if 'open' in idx:
        substrings = idx.split('/open')[:-1]
        opened = [subpath[-2:] for subpath in substrings]

    return opened


In [5]:
starting_vertex = 'AA'
starting_time = 30
paths = defaultdict(List[State])

starting_state = State(starting_vertex, False, starting_time, 0)
paths['AA'] = [starting_state]
final_states = {}
vertex_best_states = {'AA': [('AA', starting_state)]}
while len(paths):

    for idx, path in paths.copy().items():
        current_state = path[-1]
        if current_state.time>0:

            choices = graph.vertex_neigh(current_state.vertex).copy()
            if current_state.to_open:
                choices += ['open']

            for choice in choices:
                if choice == 'open':
                    assert current_state.to_open
                    new_flow = current_state.flow+(current_state.time-1)*graph.vertices[current_state.vertex].flow
                    new_state = State(vertex=current_state.vertex, to_open=False, time=current_state.time-1, flow=new_flow)
                else:
                    vertex_flow = graph.vertices[choice].flow
                    opened_vertices = extract_opened_vertices(idx)
                    to_open = vertex_flow>0 and (choice not in opened_vertices)
                    new_state = State(vertex=choice, to_open=to_open, time=current_state.time-1, flow=current_state.flow)

                new_idx = idx+'/'+choice
                if new_state.vertex in vertex_best_states.keys():
                    # print(f'at path {idx} given choice {choice} trying to add state {new_state}')
                    new_best, added, to_remove = add_state(vertex_best_states[new_state.vertex], (new_idx, new_state))
                    vertex_best_states[new_state.vertex] = new_best
                    if added:
                        new_path = path + [new_state]
                        paths[new_idx] = new_path
                    for r in to_remove:
                        if r in paths.keys():
                            paths.pop(r)
                else:
                    vertex_best_states[new_state.vertex] = [(new_idx, new_state)]
                    new_path = path + [new_state]
                    paths[new_idx] = new_path
            if idx in paths.keys():
                paths.pop(idx)
        else:
            final_states[idx] = current_state
            paths.pop(idx)
    print(f'at {current_state.time} there are {len(paths)} paths')

print(len(final_states))

best_state = (0,'AA')
for key, state in final_states.items():
    if state.flow>best_state[0]:
        best_state = (state.flow, key)

print(best_state)

at 30 there are 5 paths
at 29 there are 5 paths
at 28 there are 5 paths
at 27 there are 25 paths
at 26 there are 29 paths
at 25 there are 33 paths
at 24 there are 41 paths
at 23 there are 62 paths
at 22 there are 55 paths
at 21 there are 50 paths
at 20 there are 55 paths
at 19 there are 54 paths
at 18 there are 48 paths
at 17 there are 46 paths
at 16 there are 47 paths
at 15 there are 46 paths
at 14 there are 39 paths
at 13 there are 50 paths
at 12 there are 40 paths
at 11 there are 32 paths
at 10 there are 40 paths
at 9 there are 46 paths
at 8 there are 43 paths
at 7 there are 36 paths
at 6 there are 45 paths
at 5 there are 39 paths
at 4 there are 42 paths
at 3 there are 43 paths
at 2 there are 40 paths
at 1 there are 40 paths
at 0 there are 0 paths
40
(1944, 'AA/XM/GA/MH/open/VS/QW/open/KJ/ZU/open/DN/UX/NT/open/RZ/KF/open/VI/FF/open/BA/XY/open/WZ/NQ/open/YQ/SZ/FF/UL')


In [6]:
# @total_ordering
class DoubleState(object):
    def __init__(self, vertex1, vertex2, to_open1, to_open2, time, flow):
        self.vertex1 = vertex1
        self.vertex2 = vertex2
        self.to_open1 = to_open1
        self.to_open2 = to_open2
        self.time = time
        self.flow = flow

    def __eq__(self, other):
        return (self.flow == other.flow and self.vertex1==other.vertex1 and self.vertex2==other.vertex2 and self.time==other.time and self.to_open1==other.to_open1 and self.to_open2==other.to_open2)

    # def __lt__(self, other):
    #     return self.flow < other.flow

    def __repr__(self) -> str:
    
        return f'{self.vertex}_{self.to_open}_F{self.flow}_T{self.time}'

    

In [7]:
def is_better_double(state, other):
    assert state.vertex1 == other.vertex1
    assert state.vertex2 == other.vertex2
    # if state.flow>=other.flow and state.time>=other.time and (state.to_open1 or (not other.to_open1) or state.to_open2 or (not other.to_open2)):
    #     if (state.flow>other.flow) or (state.time>other.time) or (state.to_open1 and (not other.to_open1)) or (state.to_open2 and (not other.to_open2)):
    if state.flow>=other.flow and state.time>=other.time:
        if (state.flow>other.flow) or (state.time>other.time):
            return 2
        else:
            return 1
    else:
        return 0
        
def add_double_state(states, new_state):

    betters = [is_better_double(new_state[1], s[1]) for s in states]
    worser = [is_better_double(s[1], new_state[1]) for s in states]
    removed = []
    best_states = []
    to_add = False
    for i,state in enumerate(states):
        if betters[i]<2:
            best_states.append(state)
        else:
            # print(f'removing {state[1]} because {new_state[1]} is better')
            removed.append(state[0])
    if max(worser)<2:
        best_states.append(new_state)
        to_add = True
    return best_states, to_add, removed

def extract_opened_vertices2(idx):
    opened = []
    if 'open' in idx:
        for step in idx.split('/'):
            moves = step.split('-')
            if 'open' in moves[0]:
                opened.append(moves[0].replace('open',''))
            if 'open' in moves[1]:
                opened.append(moves[1].replace('open',''))
    assert 'open' not in opened
    return opened


In [8]:
starting_vertex = 'AA'
starting_time = 26
paths = defaultdict(List[DoubleState])

starting_state = DoubleState(starting_vertex, starting_vertex, False, False, starting_time, 0)
paths['AA-AA'] = [starting_state,starting_state]
final_states = {}
vertex_best_states = {'AA-AA': [('AA-AA', starting_state)]}
while len(paths):

    for idx, path in paths.copy().items():
        current_state = path[-1]
        if current_state.time>0:

            choices1 = graph.vertex_neigh(current_state.vertex1.replace('open','')).copy()
            choices2 = graph.vertex_neigh(current_state.vertex2.replace('open','')).copy()
            if current_state.to_open1:
                choices1 += [f'{current_state.vertex1}open']
            if current_state.to_open2:
                choices2 += [f'{current_state.vertex2}open']
            opened_vertices = extract_opened_vertices2(idx)

            for choice1 in choices1:
                for choice2 in choices2:
                    if current_state.vertex1==current_state.vertex2 and 'open' in choice1 and 'open' in choice2:
                        continue
                    new_flow = current_state.flow
                    if 'open' in choice1:
                        assert 'open' not in current_state.vertex1
                        new_flow += (current_state.time-1)*graph.vertices[current_state.vertex1].flow
                        new_to_open1 = False
                    else:
                        vertex_flow = graph.vertices[choice1].flow
                        new_to_open1 = vertex_flow>0 and (choice1 not in opened_vertices)
                    if 'open' in choice2:
                        assert 'open' not in current_state.vertex2
                        new_flow += (current_state.time-1)*graph.vertices[current_state.vertex2].flow
                        new_to_open2 = False
                    else:
                        vertex_flow = graph.vertices[choice2].flow
                        new_to_open2 = vertex_flow>0 and (choice2 not in opened_vertices)

                    choices = [choice1,choice2]
                    to_opens = [new_to_open1, new_to_open2]
                    i1 = np.argmin([choice1,choice2])
                    
                    state_key = f'{choices[i1]}-{choices[1-i1]}'
                    new_state = DoubleState(choices[i1],choices[1-i1], to_opens[i1], to_opens[1-i1], current_state.time-1, new_flow)

                    new_idx = idx+'/'+state_key
                    if state_key in vertex_best_states.keys():
                        # print(f'at path {idx} given choice {choice} trying to add state {new_state}')
                        new_best, added, to_remove = add_double_state(vertex_best_states[state_key], (new_idx, new_state))
                        vertex_best_states[state_key] = new_best
                        if added:
                            new_path = path + [new_state]
                            paths[new_idx] = new_path
                        for r in to_remove:
                            if r in paths.keys():
                                paths.pop(r)
                    else:
                        vertex_best_states[state_key] = [(new_idx, new_state)]
                        new_path = path + [new_state]
                        paths[new_idx] = new_path
            if idx in paths.keys():
                paths.pop(idx)
        else:
            final_states[idx] = current_state
            paths.pop(idx)
    print(f'at {current_state.time} there are {len(paths)} paths')

print(len(final_states))

best_state = (0,'AA')
for key, state in final_states.items():
    if state.flow>best_state[0]:
        best_state = (state.flow, key)

print(best_state)

at 26 there are 15 paths
at 25 there are 40 paths
at 24 there are 165 paths
at 23 there are 1945 paths
at 22 there are 7498 paths
at 21 there are 25421 paths
at 20 there are 40194 paths
at 19 there are 39137 paths
at 18 there are 7731 paths
at 17 there are 7740 paths
at 16 there are 5449 paths
at 15 there are 4319 paths
at 14 there are 4834 paths
at 13 there are 6809 paths
at 12 there are 10548 paths
at 11 there are 12722 paths
at 10 there are 13347 paths
at 9 there are 9181 paths
at 8 there are 11032 paths
at 7 there are 12964 paths
at 6 there are 16406 paths
at 5 there are 20551 paths
at 4 there are 28861 paths
at 3 there are 22570 paths
at 2 there are 19724 paths
at 1 there are 31275 paths
at 0 there are 0 paths
31275
(2679, 'AA-AA/HR-XM/GA-SA/MH-QM/MHopen-QMopen/SL-VS/DG-QW/QWopen-XY/HN-XYopen/VC-WZ/NQ-VCopen/NQopen-UQ/IE-KD/JK-ZU/SX-ZUopen/DN-SXopen/UW-UX/EU-NT/EUopen-NTopen/DX-RZ/KF-QL/KFopen-QLopen/KQ-VI/FF-TZ/FFopen-TZopen/KQ-UL/FF-TZ')


In [9]:
print(best_state)

print(extract_opened_vertices2(best_state[1]))

(2679, 'AA-AA/HR-XM/GA-SA/MH-QM/MHopen-QMopen/SL-VS/DG-QW/QWopen-XY/HN-XYopen/VC-WZ/NQ-VCopen/NQopen-UQ/IE-KD/JK-ZU/SX-ZUopen/DN-SXopen/UW-UX/EU-NT/EUopen-NTopen/DX-RZ/KF-QL/KFopen-QLopen/KQ-VI/FF-TZ/FFopen-TZopen/KQ-UL/FF-TZ')
['MH', 'QM', 'QW', 'XY', 'VC', 'NQ', 'ZU', 'SX', 'EU', 'NT', 'KF', 'QL', 'FF', 'TZ']


In [10]:
(2563, 'AA-AA/XM-SK/GA-IK/MH-KF/open-open/VS-VI/QW-FF/open-open/KJ-BA/ZU-XY/open-open/DN-WZ/UX-NQ/NT-open/open-KD/WU-JK/NW-SX/ED-open/MO-UW/QM-EU/open-open/DW-DX/QL-QL/KQ-open/TZ-KQ/open-QL/KQ-KQ')
['MH', 'KF', 'QW', 'FF', 'ZU', 'XY', 'NQ', 'NT', 'SX', 'QM', 'EU', 'QL', 'TZ']

(3055, 'AA-AA/XM-HR/SA-GA/MH-QM/openMH-openQM/VS-SL/DG-QW/XY-openQW/openQW-WZ/HN-NQ/VC-openNQ/openNQ-HN/VC-WZ/openVC-XY/UQ-openXY/IE-BA/FF-ZU/openFF-openZU/VI-KJ/QW-KF/PG-openQW/TY-KJ/QW-NW/WU-openQW/KJ-NT/QW-openNT/WU-KJ')
['MH', 'QM', 'QW', 'QW', 'NQ', 'NQ', 'VC', 'XY', 'FF', 'ZU', 'QW', 'QW', 'NT']

(2679, 'AA-AA/HR-XM/GA-SA/MH-QM/MHopen-QMopen/SL-VS/DG-QW/QWopen-XY/HN-XYopen/VC-WZ/NQ-VCopen/NQopen-UQ/IE-KD/JK-ZU/SX-ZUopen/DN-SXopen/UW-UX/EU-NT/EUopen-NTopen/DX-RZ/KF-QL/KFopen-QLopen/KQ-VI/FF-TZ/FFopen-TZopen/KQ-UL/FF-TZ')
['MH', 'QM', 'QW', 'XY', 'VC', 'NQ', 'ZU', 'SX', 'EU', 'NT', 'KF', 'QL', 'FF', 'TZ']




['MH', 'QM', 'QW', 'QW', 'NQ', 'NQ', 'VC', 'XY', 'FF', 'ZU', 'QW', 'QW', 'NT']